In [1]:
import dgl
import time
import torch as th
import numpy as np
import torch.nn.functional as F

from load_data import load_ogb
from model import SAGE, GAT, get_eigen
from train_eval import train_ft_embed

Using backend: pytorch


This notebook shows the performance of GraphSage on the OGBN-products graph with trainable embeddings on the nodes. We use scaled eigenvectors to initialize the node embeddings. The logic here is that if eigenvectors can serve as positional node embeddings, fine-tuning the node embeddings should give us even better performance. In the experiment below, we tune the node embeddings with a much smaller learning rate than the GNN model.

In [2]:
g, num_labels = load_ogb('ogbn-products')
g.ndata.pop('features')

in_feats = 128
eigen_vals, eigen_vecs = get_eigen(g, in_feats, 'ogbn-products')
nfeat = th.tensor(eigen_vecs * np.sqrt(eigen_vals).reshape((1, len(eigen_vals))), dtype=th.float32)
labels = g.ndata.pop('labels')

hyperparams = {
    'batch_size': 1024,
    'num_workers': 0,
    'num_hidden': 256,
    'num_layers': 3,
    'dropout': 0.5,
    'lr': 0.003,
    'sparse_lr': 0.0003,
    'num_epochs': 20,
    'fanouts': [15,10,5],
    'eval_batch_size': 10000
}
device = th.device('cuda:5')

# Pack data
data = g, nfeat, labels
# Define model
model = SAGE(in_feats, hyperparams['num_hidden'], num_labels,
             hyperparams['num_layers'], F.relu, hyperparams['dropout'])
train_ft_embed(model, data, hyperparams, device, 1)

load ogbn-products
finish loading ogbn-products
finish constructing ogbn-products


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:219.)
  


embedding is in  cuda:5


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Step 00000 | Loss 3.8511 | Train Acc 0.0176 | Speed (samples/sec) nan | GPU 4989.3 MB
Epoch 00000 | Step 00020 | Loss 1.8255 | Train Acc 0.5273 | Speed (samples/sec) 346.1535 | GPU 5001.3 MB
Epoch 00000 | Step 00040 | Loss 0.8426 | Train Acc 0.8076 | Speed (samples/sec) 362.6535 | GPU 5005.0 MB
Epoch 00000 | Step 00060 | Loss 0.6259 | Train Acc 0.8604 | Speed (samples/sec) 368.2415 | GPU 5005.0 MB
Epoch 00000 | Step 00080 | Loss 0.5090 | Train Acc 0.8809 | Speed (samples/sec) 363.2799 | GPU 5005.0 MB
Epoch 00000 | Step 00100 | Loss 0.4553 | Train Acc 0.8984 | Speed (samples/sec) 360.6319 | GPU 5005.5 MB
Epoch 00000 | Step 00120 | Loss 0.3845 | Train Acc 0.8984 | Speed (samples/sec) 361.3783 | GPU 5010.6 MB
Epoch 00000 | Step 00140 | Loss 0.4392 | Train Acc 0.8945 | Speed (samples/sec) 362.8149 | GPU 5010.6 MB
Epoch 00000 | Step 00160 | Loss 0.3487 | Train Acc 0.9102 | Speed (samples/sec) 360.3278 | GPU 5010.6 MB
Epoch 00000 | Step 00180 | Loss 0.3094 | Train Acc 0.9219 | 

...

Val acc 0.8999, Test Acc: 0.7602
Epoch 00019 | Step 00000 | Loss 0.0011 | Train Acc 1.0000 | Speed (samples/sec) 369.4070 | GPU 5013.3 MB
Epoch 00019 | Step 00020 | Loss 0.0171 | Train Acc 0.9961 | Speed (samples/sec) 369.6732 | GPU 5013.3 MB
Epoch 00019 | Step 00040 | Loss 0.0045 | Train Acc 0.9980 | Speed (samples/sec) 369.9219 | GPU 5013.3 MB
Epoch 00019 | Step 00060 | Loss 0.0022 | Train Acc 0.9990 | Speed (samples/sec) 370.1807 | GPU 5013.3 MB
Epoch 00019 | Step 00080 | Loss 0.0019 | Train Acc 0.9990 | Speed (samples/sec) 370.3640 | GPU 5013.3 MB
Epoch 00019 | Step 00100 | Loss 0.0020 | Train Acc 1.0000 | Speed (samples/sec) 370.5634 | GPU 5013.3 MB
Epoch 00019 | Step 00120 | Loss 0.0037 | Train Acc 0.9980 | Speed (samples/sec) 370.8572 | GPU 5013.3 MB
Epoch 00019 | Step 00140 | Loss 0.0017 | Train Acc 0.9990 | Speed (samples/sec) 371.0589 | GPU 5013.3 MB
Epoch 00019 | Step 00160 | Loss 0.0068 | Train Acc 0.9990 | Speed (samples/sec) 371.2727 | GPU 5013.3 MB
Epoch 00019 | Step 001

  0%|          | 0/245 [00:00<?, ?it/s]

Epoch Time(s): 475.8958


100%|██████████| 245/245 [01:09<00:00,  3.52it/s]


Val acc 0.8997, Test Acc: 0.7566
Avg epoch time: 670.0859654108683
Best val acc 0.9164, best test acc: 0.7900
